In [1]:
import os

import numpy as np
import matplotlib.pyplot as plt
import torch
from sklearn.datasets import load_digits
from sklearn import datasets
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn

from pytorch_model_summary import summary

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


**DISCLAIMER**

The presented code is not optimized, it serves an educational purpose. It is written for CPU, it uses only fully-connected networks and an extremely simplistic dataset. However, it contains all components that can help to understand how RealNVP works, and it should be rather easy to extend it to more sophisticated models. This code could be run almost on any laptop/PC, and it takes a couple of minutes top to get the result.

### Dataset

In this example, we go wild and use a dataset that is simpler than MNIST! We use a scipy dataset called Digits. It consists of ~1500 images of size 8x8, and each pixel can take values in $\{0, 1, \ldots, 16\}$.

The goal of using this dataset is that everyone can run it on a laptop, without any gpu etc.

In [2]:
class Digits(Dataset):
    """Scikit-Learn Digits dataset."""

    def __init__(self, mode='train', transforms=None):
        digits = load_digits()
        if mode == 'train':
            self.data = digits.data[:1000].astype(np.float32)
        elif mode == 'val':
            self.data = digits.data[1000:1350].astype(np.float32)
        else:
            self.data = digits.data[1350:].astype(np.float32)

        self.transforms = transforms

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data[idx]
        if self.transforms:
            sample = self.transforms(sample)
        return sample

### RealNVP code

Please see the blogpost for details.

In [ ]:
initialize_orthogonal = lambda shape: nn.init.orthogonal_((shape))

In [9]:
class RealNVP(nn.Module):
    
    def __init__(self, nets, nett, num_flows, prior, D=2, dequantization=True):
        super(RealNVP, self).__init__()
        
        print('RealNVP by JT.')
        
        self.dequantization = dequantization
        
        self.prior = prior
        self.t = torch.nn.ModuleList([nett() for _ in range(num_flows)])
        self.s = torch.nn.ModuleList([nets() for _ in range(num_flows)])
        # TODO Q3: add orthogonal matrix
        # orthogonal = lambda shape: nn.init.orthogonal_(torch.empty(shape))
        # self.r = nn.ParameterList([nn.Parameter(orthogonal((D,D))) for _ in range(num_flows)])
        self.r = torch.nn.ModuleList([nn.Linear(D, D, bias=False) for _ in range(num_flows)])
        for layer in self.r:
            torch.nn.init.orthogonal_(layer.weight)
        


         
        
        self.num_flows = num_flows
        
        self.D = D

    def coupling(self, x, index, forward=True):
        # x: input, either images (for the first transformation) or outputs from the previous transformation
        # index: it determines the index of the transformation
        # forward: whether it is a pass from x to y (forward=True), or from y to x (forward=False)
        
     
        (xa, xb) = torch.chunk(x, 2, 1) # split into 2 parts in dim=1


        
        s = self.s[index](xa)
        t = self.t[index](xa)
        
        if forward:
            #yb = f^{-1}(x)
            yb = (xb - t) * torch.exp(-s)
        else:
            #xb = f(y)
            yb = torch.exp(s) * xb + t
        
        return torch.cat((xa, yb), 1), s # recombine two parts along dim =1, return scale

    def permute(self, x):
        
        return x.flip(1)
    def permute(self,x,index,forward = True):
        # TODO: Q3
        if forward:
            #print(x.shape)
            #print(self.r[index].shape)
            x = x @ self.r[index].weight
            r = torch.det(self.r[index].weight).abs().log()
        else:
            x = x @ self.r[index].weight.inverse()
            r = -torch.det(self.r[index].weight).abs().log()
        return x,r

    def f(self, x):
        log_det_J, z = x.new_zeros(x.shape[0]), x
        for i in range(self.num_flows):
            z, s = self.coupling(z, i, forward=True)
            z, r = self.permute(z,i,forward=True)
            
            log_det_J = log_det_J + r - s.sum(dim=1)
             

        return z, log_det_J

    def f_inv(self, z):
        x = z
        for i in reversed(range(self.num_flows)):
            #x = self.permute(x)
            x, _ = self.permute(x,i,forward =False)
            x, _ = self.coupling(x, i, forward=False)
            

        return x

    def forward(self, x, reduction='avg'):
        z, log_det_J = self.f(x)
        if reduction == 'sum':
            return -(self.prior.log_prob(z) + log_det_J).sum()
        else:
            return -(self.prior.log_prob(z) + log_det_J).mean()

    def sample(self, batchSize):
        z = self.prior.sample((batchSize, self.D)) #每一个维度的值有相关性，考虑prior是多元分布
        z = z[:, 0, :] # 对z进行操作，抛弃了其中一个维度
        x = self.f_inv(z)
        return x.view(-1, self.D)

Thinking…

### Auxiliary functions: training, evaluation, plotting

It's rather self-explanatory, isn't it?

In [10]:
def evaluation(test_loader, name=None, model_best=None, epoch=None):
    # EVALUATION
    if model_best is None:
        # load best performing model
        model_best = torch.load(name + '.model')

    model_best.eval()
    loss = 0.
    N = 0.
    for indx_batch, test_batch in enumerate(test_loader):
        if hasattr(model, 'dequantization'):
            if model.dequantization:
                test_batch = test_batch + (1. - torch.rand(test_batch.shape))/2.
        loss_t = model_best.forward(test_batch, reduction='sum')
        loss = loss + loss_t.item()
        N = N + test_batch.shape[0]
    loss = loss / N

    if epoch is None:
        print(f'FINAL LOSS: nll={loss}')
    else:
        print(f'Epoch: {epoch}, val nll={loss}')

    return loss


def samples_real(name, test_loader):
    # REAL-------
    num_x = 4
    num_y = 4
    x = next(iter(test_loader)).detach().numpy()

    fig, ax = plt.subplots(num_x, num_y)
    for i, ax in enumerate(ax.flatten()):
        plottable_image = np.reshape(x[i], (8, 8))
        ax.imshow(plottable_image, cmap='gray')
        ax.axis('off')

    plt.savefig(name+'_real_images.pdf', bbox_inches='tight')
    plt.close()


def samples_generated(name, data_loader, extra_name=''):
    x = next(iter(data_loader)).detach().numpy()

    # GENERATIONS-------
    model_best = torch.load(name + '.model')
    model_best.eval()

    num_x = 4
    num_y = 4
    x = model_best.sample(num_x * num_y)
    x = x.detach().numpy()

    fig, ax = plt.subplots(num_x, num_y)
    for i, ax in enumerate(ax.flatten()):
        plottable_image = np.reshape(x[i], (8, 8))
        ax.imshow(plottable_image, cmap='gray')
        ax.axis('off')

    plt.savefig(name + '_generated_images' + extra_name + '.pdf', bbox_inches='tight')
    plt.close()


def plot_curve(name, nll_val):
    plt.plot(np.arange(len(nll_val)), nll_val, linewidth='3')
    plt.xlabel('epochs')
    plt.ylabel('nll')
    plt.savefig(name + '_nll_val_curve.pdf', bbox_inches='tight')
    plt.close()

In [11]:
def training(name, max_patience, num_epochs, model, optimizer, training_loader, val_loader):
    nll_val = []
    best_nll = 1000.
    patience = 0

    # Main loop
    for e in range(num_epochs):
        # TRAINING
        model.train()
        for indx_batch, batch in enumerate(training_loader):
            if hasattr(model, 'dequantization'):
                if model.dequantization:
                    batch = batch + (1. - torch.rand(batch.shape))/2.
            loss = model.forward(batch)

            optimizer.zero_grad()
            loss.backward(retain_graph=True)
            optimizer.step()

        # Validation
        loss_val = evaluation(val_loader, model_best=model, epoch=e)
        nll_val.append(loss_val)  # save for plotting

        if e == 0:
            print('saved!')
            torch.save(model, name + '.model')
            best_nll = loss_val
        else:
            if loss_val < best_nll:
                print('saved!')
                torch.save(model, name + '.model')
                best_nll = loss_val
                patience = 0

                samples_generated(name, val_loader, extra_name="_epoch_" + str(e))
            else:
                patience = patience + 1

        if patience > max_patience:
            break

    nll_val = np.asarray(nll_val)

    return nll_val

### Initialize dataloaders

In [12]:
train_data = Digits(mode='train')
val_data = Digits(mode='val')
test_data = Digits(mode='test')

training_loader = DataLoader(train_data, batch_size=64, shuffle=True)
val_loader = DataLoader(val_data, batch_size=64, shuffle=False)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False)

result_dir = 'results/'
if not(os.path.exists(result_dir)):
    os.mkdir(result_dir)
name = 'realnvp'

### Hyperparams

In [13]:
D = 64   # input dimension
M = 256  # the number of neurons in scale (s) and translation (t) nets

lr = 1e-3 # learning rate
num_epochs = 100 # max. number of epochs
max_patience = 20 # an early stopping is used, if training doesn't improve for longer than 20 epochs, it is stopped

### Initialize RealNVP

In [14]:
# The number of invertible transformations
num_flows = 8

# scale (s) network i.e.variance
nets = lambda: nn.Sequential(nn.Linear(D // 2, M), nn.LeakyReLU(),
                             nn.Linear(M, M), nn.LeakyReLU(),
                             nn.Linear(M, D // 2), nn.Tanh())

# translation (t) network i.e. mean
nett = lambda: nn.Sequential(nn.Linear(D // 2, M), nn.LeakyReLU(),
                             nn.Linear(M, M), nn.LeakyReLU(),
                             nn.Linear(M, D // 2))

# Prior (a.k.a. the base distribution): Gaussian # init with independence as covariance = 0
prior = torch.distributions.MultivariateNormal(torch.zeros(D), torch.eye(D))
# Init RealNVP
model = RealNVP(nets, nett, num_flows, prior, D=D, dequantization=True)
# Print the summary (like in Keras)
print(summary(model, torch.zeros(1, 64), show_input=False, show_hierarchical=False))

RealNVP by JT.
-----------------------------------------------------------------------
      Layer (type)        Output Shape         Param #     Tr. Param #
          Linear-1            [1, 256]           8,448           8,448
       LeakyReLU-2            [1, 256]               0               0
          Linear-3            [1, 256]          65,792          65,792
       LeakyReLU-4            [1, 256]               0               0
          Linear-5             [1, 32]           8,224           8,224
            Tanh-6             [1, 32]               0               0
          Linear-7            [1, 256]           8,448           8,448
       LeakyReLU-8            [1, 256]               0               0
          Linear-9            [1, 256]          65,792          65,792
      LeakyReLU-10            [1, 256]               0               0
         Linear-11             [1, 32]           8,224           8,224
         Linear-12            [1, 256]           8,448       

### Let's play! Training

In [15]:
# OPTIMIZER
optimizer = torch.optim.Adamax([p for p in model.parameters() if p.requires_grad == True], lr=lr)

In [16]:
# Training procedure
nll_val = training(name=result_dir + name, max_patience=max_patience, num_epochs=num_epochs, model=model, optimizer=optimizer,
                       training_loader=training_loader, val_loader=val_loader)

Epoch: 0, val nll=316.81866629464287
saved!
Epoch: 1, val nll=281.71328683035716
saved!
Epoch: 2, val nll=266.8484612165179
saved!
Epoch: 3, val nll=257.83332589285715
saved!
Epoch: 4, val nll=250.95230050223213
saved!
Epoch: 5, val nll=245.33097237723214
saved!
Epoch: 6, val nll=240.44608537946428
saved!
Epoch: 7, val nll=235.92285714285714
saved!
Epoch: 8, val nll=232.2586049107143
saved!
Epoch: 9, val nll=228.74977957589286
saved!
Epoch: 10, val nll=225.58858258928572
saved!
Epoch: 11, val nll=222.46487723214287
saved!
Epoch: 12, val nll=219.8542494419643
saved!
Epoch: 13, val nll=217.26418805803573
saved!
Epoch: 14, val nll=214.88447963169642
saved!
Epoch: 15, val nll=212.73116071428572
saved!
Epoch: 16, val nll=210.5095591517857
saved!
Epoch: 17, val nll=208.50363420758927
saved!
Epoch: 18, val nll=206.70188616071428
saved!
Epoch: 19, val nll=204.9732924107143
saved!
Epoch: 20, val nll=203.27869977678571
saved!
Epoch: 21, val nll=201.77747628348214
saved!
Epoch: 22, val nll=200.41

In [ ]:
class Model(nn.Module):
    def __init__(self, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)
        self.a = nn.ParameterList([nn.Parameter(torch.rand(10), requires_grad=True)])


m = Model()
b = m.a[0].mean()
b.backward()

print(m.a[0].grad)

In [ ]:
print(model.r[1].grad)

In [ ]:
test_loss = evaluation(name=result_dir + name, test_loader=test_loader)
f = open(result_dir + name + '_test_loss.txt', "w")
f.write(str(test_loss))
f.close()

samples_real(result_dir + name, test_loader)

plot_curve(result_dir + name, nll_val)

In [ ]:
import torch

# 创建一个示例张量
x = torch.tensor([[1, 2, 3], [4, 5, 6], [7, 8, 9]])

# 选择一个维度（例如，维度1）来压缩
dim_to_compress = 1



# 使用布尔索引来排除奇数值
compressed_x = x[:, x[dim_to_compress] % 2 == 0]

print(compressed_x)


In [ ]:
tensor = torch.arange(x.shape[1])
xa = x[:, tensor % 2 == 0]
       #  xb = x[:, x[dim_to_compress] % 2 == 1]

In [ ]:
xa

In [ ]:
import torch
import torch.nn as nn

# 创建一个2x2的张量，用于正交矩阵初始化
matrix = torch.empty(2, 2)

# 使用orthogonal_方法初始化为正交矩阵
nn.init.orthogonal_(matrix)

print(matrix)

In [ ]:
y = torch.tensor([[1,2],[3,4]])
z = y*matrix


In [ ]:
torch.det(matrix)

In [ ]:
r = [initialize_orthogonal((64, 64)) for _ in range(2)]

In [ ]:
torch.det(r[0]).item()